In [1]:
%load_ext autoreload
%autoreload 2

# Dynamic Programming

In [2]:
from othello.OthelloGame import OthelloGame as Game
from othello.OthelloGame import display as displayGame
from matplotlib import pyplot as plt
from matplotlib import patches
import numpy as np
from tree_search_algs import bfs_cannonical

## Tree Search to find all states
Note: we are finding all states given that player 1 (white) starts playing. 
States of player 2 are expressed in cannonical form. 

In [3]:
n = 4
game = Game(n)
board = game.getInitBoard()

In [70]:
# Find all states of game doing a search tree if player 1 starts
first_player = 1
%time states_actions_player_1 = bfs_cannonical(game, board, first_player) # (1 white)

CPU times: user 59.1 s, sys: 1.07 s, total: 1min
Wall time: 1min


In [71]:
# Find all states of game doing a search tree if player 2 starts
first_player = -1
%time states_actions_player_2 = bfs_cannonical(game, board, first_player) #(-1 black)

CPU times: user 59.2 s, sys: 491 ms, total: 59.7 s
Wall time: 59.6 s


In [72]:
states_actions = {**states_actions_player_1, **states_actions_player_2}

In [73]:
print('Number of states:', len(states_actions_player_1))
print('Number of states:', len(states_actions_player_2))

Number of states: 53651
Number of states: 53651


In [74]:
print('Number of states:', len(states_actions))

Number of states: 91282


In [75]:
np.save('states_actions', states_actions)

In [76]:
# Size of the file. Only works if linux or mac. Try dir for windows
! ls -lah states_actions.npy

-rw-r--r--  1 julianganzabal  staff   144M Mar  6 12:01 states_actions.npy


In [77]:
state = list(states_actions.keys())[0]
print('Example of state:')
print(state)
print()
print('Actions, rewards and next nodes of state:')
for k, v in states_actions[state].items():
    print(k, v)

Example of state:
(0, 0, 0, 0, 0, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0)

Actions, rewards and next nodes of state:
1 {'winner': 0, 'next_node': (0, -1, 0, 0, 0, -1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)}
4 {'winner': 0, 'next_node': (0, 0, 0, 0, -1, -1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)}
11 {'winner': 0, 'next_node': (0, 0, 0, 0, 0, 1, -1, 0, 0, -1, -1, -1, 0, 0, 0, 0)}
14 {'winner': 0, 'next_node': (0, 0, 0, 0, 0, 1, -1, 0, 0, -1, -1, 0, 0, 0, -1, 0)}


### Questions:

In [78]:
# Board after player 1 plays action 1
board_1 = np.array(
[[0,  1,  0,  0],
[ 0,  1,  1,  0],
[ 0,  1, -1,  0],
[ 0,  0,  0,  0]])
states_actions[tuple(board_1.reshape(-1))]

KeyError: (0, 1, 0, 0, 0, 1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0)

why the above is not a valid state? Which is the state for the Board after player 1 plays action 1?

## Generate a uniform stochastic policy
It is used as initial policy to test stochastic policy evalution 

In [5]:
states_actions = np.load('states_actions.npy').item()

In [6]:
from dynamic_programming import generate_uniform_stochastic_policy
pi = generate_uniform_stochastic_policy(states_actions)

for i in range(2):
    state = list(pi.keys())[i]
    print('Example of state:')
    print(state)
    print('Actions, Probabilities:')
    for k, v in pi[state].items():
        print(k, v)
    print('------------------')

Example of state:
(0, 0, 0, 0, 0, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0)
Actions, Probabilities:
1 0.25
4 0.25
11 0.25
14 0.25
------------------
Example of state:
(0, -1, 0, 0, 0, -1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)
Actions, Probabilities:
0 0.3333333333333333
2 0.3333333333333333
8 0.3333333333333333
------------------


In [7]:
pi[(0, 0, 0, 0, 0, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

{1: 0.25, 4: 0.25, 11: 0.25, 14: 0.25}

## Policy evaluation test
### Stochastic policy evaluation

In [13]:
from dynamic_programming import policy_evaluation, stochastic_policy_eval_step
# stochastic_policy_eval_step does policy_evaluation using probabilities. Check library

In [81]:
# Run it multiple times to check it takes different number of iterations to converge
stochastic_pi = generate_uniform_stochastic_policy(states_actions)
V_stochastic, iters = policy_evaluation(stochastic_policy_eval_step, 
                             states_actions, 
                             stochastic_pi, 1e-8, verbose=1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 


In [82]:
print('10 examples of Value function:')
i = 0 
for k, v in V_stochastic.items():
    print(k, v)
    if i == 10:
        break
    i += 1

10 examples of Value function:
(0, 0, 0, 0, 0, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0) -0.19926756827206293
(0, -1, 0, 0, 0, -1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0) 0.19926756827206293
(0, 0, 0, 0, -1, -1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0) 0.19926756827206293
(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, -1, -1, 0, 0, 0, 0) 0.1992675682720629
(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, -1, 0, 0, 0, -1, 0) 0.19926756827206293
(-1, 1, 0, 0, 0, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0) -0.7334317552457347
(0, 1, -1, 0, 0, 1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0) 0.10847170895012459
(0, 1, 0, 0, 0, 1, 1, 0, -1, -1, -1, 0, 0, 0, 0, 0) 0.027157341479421185
(-1, 0, 0, 0, 1, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0) -0.7334317552457347
(0, 0, -1, 0, 1, 1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0) 0.027157341479421192
(0, 0, 0, 0, 1, 1, 1, 0, -1, -1, -1, 0, 0, 0, 0, 0) 0.10847170895012459


### Deterministic policy evaluation

In [8]:
from dynamic_programming import generate_deterministic_policy, deterministic_policy_eval_step

In [10]:
deterministic_pi = generate_deterministic_policy(states_actions)
for i in range(2):
    state = list(pi.keys())[i]
    print('Example of state:', state)
    print('"best" action:', deterministic_pi[state])
    print('------------------')

Example of state: (0, 0, 0, 0, 0, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0)
"best" action: 11
------------------
Example of state: (0, -1, 0, 0, 0, -1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)
"best" action: 8
------------------


In [11]:
deterministic_pi[(0, 0, 0, 0, 0, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

11

In [14]:
# Run it multiple times to check it always takes the same number of iterations
V_deterministic, iters = policy_evaluation(deterministic_policy_eval_step, 
                             states_actions, 
                             deterministic_pi, 1e-8, verbose=1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 


In [86]:
print('10 examples of Value function:')
i = 0 
for k, v in V_deterministic.items():
    print(k, v)
    if i == 10:
        break
    i += 1

10 examples of Value function:
(0, 0, 0, 0, 0, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0) -1
(0, -1, 0, 0, 0, -1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0) 1
(0, 0, 0, 0, -1, -1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0) 1
(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, -1, -1, 0, 0, 0, 0) -1
(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, -1, 0, 0, 0, -1, 0) 1
(-1, 1, 0, 0, 0, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0) -1
(0, 1, -1, 0, 0, 1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0) 1e-06
(0, 1, 0, 0, 0, 1, 1, 0, -1, -1, -1, 0, 0, 0, 0, 0) -1
(-1, 0, 0, 0, 1, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, 0) -1
(0, 0, -1, 0, 1, 1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0) -1
(0, 0, 0, 0, 1, 1, 1, 0, -1, -1, -1, 0, 0, 0, 0, 0) 1


See values are just 1 or -1 (They could be +-0.2 for ties)

## Policy Improve

In [87]:
from dynamic_programming import policy_improve, policy_iteration

In [88]:
initial_policy = generate_deterministic_policy(states_actions)
optimum_policy, optimum_V = policy_iteration(states_actions, initial_policy, verbose = 1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 
Number of differences of new policy vs old policy: 23050
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 4207
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 
Number of differences of new policy vs old policy: 1175
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 295
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 82
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 
Number of differences of new policy vs old policy: 25
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 9
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of 

In [89]:
n = 4
game = Game(n)
board = game.getInitBoard()
player = 1
print(board)

[[ 0  0  0  0]
 [ 0 -1  1  0]
 [ 0  1 -1  0]
 [ 0  0  0  0]]


In [90]:
# Empieza player_1
first_player = 1
print(optimum_V[tuple(first_player * board.reshape(-1))])

-1


The above result indicates that player 1 always looses

In [91]:
# Empieza player_2
first_player = -1
print(optimum_V[tuple(first_player * board.reshape(-1))])

-1


In [92]:
np.save('Value_func_only_winner', optimum_V)

In [93]:
np.save('pi_func_only_winner', optimum_policy)

In [94]:
! ls -lah *.npy

-rw-r--r--  1 julianganzabal  staff    86M Mar  6 11:10 Value_func_margin_reward.npy
-rw-r--r--  1 julianganzabal  staff    76M Mar  6 12:03 Value_func_only_winner.npy
-rw-r--r--  1 julianganzabal  staff    80M Feb 24 22:56 Value_func_steps_reward.npy
-rw-r--r--  1 julianganzabal  staff    85M Mar  6 11:10 pi_func_margin_reward.npy
-rw-r--r--  1 julianganzabal  staff    80M Mar  6 12:03 pi_func_only_winner.npy
-rw-r--r--  1 julianganzabal  staff    80M Feb 24 22:56 pi_func_steps_reward.npy
-rw-r--r--  1 julianganzabal  staff   144M Mar  6 12:01 states_actions.npy


# Lets play game with optimun policy

In [95]:
from othello.OthelloGame import OthelloGame as Game
from othello.OthelloGame import display as displayGame
from matplotlib import pyplot as plt
from matplotlib import patches
import numpy as np

from playing_stats import EvaluatePolicy

In [96]:
optimum_policy = np.load('pi_func_only_winner.npy').item()

In [97]:
evalPolicy = EvaluatePolicy(optimum_policy)

In [98]:
n = 4
game = Game(n)
board = game.getInitBoard()
player = 1

# Random vs Random as and Greedy vs Greedy as reference

In [63]:
def display_results(player_1_wins, player_2_wins, ties, margins, steps_array, pieces):
    print('player_1 wins:', str(int(100*player_1_wins/episodes + 0.5)) + '%')
    print('player_2 wins:', str(int(100*player_2_wins/episodes + 0.5)) +'%')
    print('ties:', str(int(100*ties/episodes + 0.5))+ '%')
    print('Max, Mean, Min margins: ', end ='')
    print(np.max(margins), np.mean(margins), np.min(margins))
    print('Max, Mean, Min steps: ', end ='')
    print(np.max(steps_array), np.mean(steps_array), np.min(steps_array))
    print('Max, Mean, Min pieces: ', end ='')
    print(np.max(pieces), np.mean(pieces), np.min(pieces))

In [64]:
episodes = 1000
player_1_wins, player_2_wins, ties, margins, steps_array, pieces = evalPolicy.get_stats(game, 
                                                board, 
                                                {1: evalPolicy.random_player, -1: evalPolicy.random_player}, 
                                                episodes)
display_results(player_1_wins, player_2_wins, ties, margins, steps_array, pieces)

player_1 wins: 38%
player_2 wins: 53%
ties: 9%
Max, Mean, Min margins: 16 -1.437 -16
Max, Mean, Min steps: 16 12.562 7
Max, Mean, Min pieces: 16 15.747 10


Second player seems to have more probability to win playing random

In [65]:
episodes = 1000
player_1_wins, player_2_wins, ties, margins, steps_array, pieces = evalPolicy.get_stats(game, 
                                                board, 
                                                {1: evalPolicy.greedy_player, -1: evalPolicy.greedy_player}, 
                                                episodes)
display_results(player_1_wins, player_2_wins, ties, margins, steps_array, pieces)

player_1 wins: 44%
player_2 wins: 49%
ties: 7%
Max, Mean, Min margins: 15 -0.671 -16
Max, Mean, Min steps: 15 12.21 7
Max, Mean, Min pieces: 16 15.517 10


## Policy plays second against random

In [66]:
episodes = 1000
player_1_wins, player_2_wins, ties, margins, steps_array, pieces = evalPolicy.get_stats(game, 
                                                board, 
                                                {1: evalPolicy.random_player, -1: evalPolicy.policy_player}, 
                                                episodes)
display_results(player_1_wins, player_2_wins, ties, margins, steps_array, pieces)

player_1 wins: 0%
player_2 wins: 100%
ties: 0%
Max, Mean, Min margins: -2 -7.144 -16
Max, Mean, Min steps: 14 12.639 10
Max, Mean, Min pieces: 16 15.844 13


Optimal policy always win as second player

## Policy plays first against random

In [12]:
episodes = 1000
player_1_wins, player_2_wins, ties, margins, steps_array, pieces = evalPolicy.get_stats(game, 
                                                board, 
                                                {1: evalPolicy.policy_player, -1: evalPolicy.random_player}, 
                                                episodes)
display_results(player_1_wins, player_2_wins, ties, margins, steps_array, pieces)

player_1 wins: 83%
player_2 wins: 18%
ties: 0%
Max, Mean, Min margins: 14 3.154 -9
Max, Mean, Min steps: 14 11.654 9
Max, Mean, Min pieces: 16 15.188 12


Obtimal policy can't win all but does much better than random player 1

## Policy plays first against greedy

In [13]:
episodes = 1000
player_1_wins, player_2_wins, ties, margins, steps_array, pieces = evalPolicy.get_stats(game, 
                                                board, 
                                                {1: evalPolicy.policy_player, -1: evalPolicy.greedy_player}, 
                                                episodes)
display_results(player_1_wins, player_2_wins, ties, margins, steps_array, pieces)

player_1 wins: 68%
player_2 wins: 32%
ties: 0%
Max, Mean, Min margins: 14 2.795 -9
Max, Mean, Min steps: 14 11.724 9
Max, Mean, Min pieces: 16 15.139 12


Also does good as player 1 against greedy